# Pandas Assignment

## Part 1

In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting

Hint! Find the highly rated movies which appeals to both genders 'M' and 'F'


In [1]:
import os
import pandas as pd
import numpy as np
from math import *

##### Read in the movie data: `pd.read_table`

In [2]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames)
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames)
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames)

    return users, ratings, movies

In [3]:
users, ratings, movies = get_movie_data()

C:\Users\datasoup\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\Users\datasoup\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
C:\Users\datasoup\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


In [4]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [7]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

C:\Users\datasoup\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


0    Toy Story
1         1995
dtype: object

In [8]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [9]:
movies.head()

,movie_id,title,genres,year,short_title
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


##### Join the tables with `pd.merge`

In [10]:
# I suppose I should join the movies with their ratings?
# Perform a left join, because we want to keep movies which don't have any ratings and
# report them/use them as movies which have not yet been rated by any users.
movies_ratings = pd.merge(left=movies,right=ratings,how='left')
movies_ratings.head()

,movie_id,title,genres,year,short_title,user_id,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,8.0,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,9.0,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,10.0,5.0,978226474.0


**Aggregate the data for number of ratings and mean rating**

In [11]:
# how should we determine the hightest rated movie?
# perhaps I can take the average rating of each movie based on it's number of ratings
movies_ratings[movies_ratings.isnull().any(axis=1)]

# we can aggregate the dataframe on movie_id, creating a new dataframe with all the original columns
# but the rating is now the mean rating for each film, and we add a new column called num_ratings,
# which tracks the number of ratings given for each film.

# define our aggregation funcitons so that we get proper names after aggregation.
def num_ratings(x):
    return len(x)
def mean_rating(x):
    return x.mean()

# group by movie_id and title, etc, letting us keep those values.
g = movies_ratings.groupby(['movie_id','title','genres','year'])
movies_agg = g.agg({'user_id':[num_ratings], 'rating':[mean_rating]})

# get rid of first level of columns
movies_agg.columns = movies_agg.columns.droplevel(0)
# turn our movie_id + title index into columns, and reindex
movies_agg.reset_index(inplace=True)
movies_agg.head()

,movie_id,title,genres,year,num_ratings,mean_rating
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,2077.0,4.146846
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,701.0,3.201141
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,478.0,3.016736
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,170.0,2.729412
4,5,Father of the Bride Part II (1995),Comedy,1995,296.0,3.006757


##### What's the highest rated movie?

We can't simply choose the movie which has the highest mean rating, because this would be a skewed 
result. Suppose a movie only had one rating which was a 5. This can't be considered for highest rated since it
only represents one sample out of the population. Obviously a movie with an average rating of 4.5 and 100 ratings
should rank higher. To solve this we need to define some way to rank a film based on the number of ratings it has 
and it's average rating.

Consider the following:

$$Rank = \frac{ratings}{rating_{avg}}$$

This gives us an *okay enough* ranking for movies. For example a movie, $A$ with $ratings = 5$ and $rating_{avg} = 4.7$ would rank lower than another movie, $B$ with $ratings = 10$ and $rating_{avg} = 4.1$

$$Rank_A = \frac{ratings_A}{rating_{avgA}} = \frac{5}{4.7} = 1.06$$

$$Rank_B = \frac{ratings_B}{rating_{avgB}} = \frac{10}{4.1} = 2.44$$

$$1.06 < 2.44$$

This works well enough for our purposes because the number of ratings is a continuous variable, whereas the mean rating is limited to the interval $[0,5]$. This means that we effectively produce another continuous variable, rank, from the number of ratings and the mean rating of each movie. Doing it the other way around would result in cases in which films with a higher rating than the number of ratings, would get higher rank than they deserve, and the rank variable would be limited to the interval $[0,5]$. We also leave the decimal portion of each rank, incase two or more movies have the same whole number portion of rank, we can choose between them in most cases.

In [12]:
# Assign a rank to the dataframe
movies_agg['rank'] = movies_agg['num_ratings'] / movies_agg['mean_rating']
movies_agg.head()

,movie_id,title,genres,year,num_ratings,mean_rating,rank
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,2077.0,4.146846,500.862533
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,701.0,3.201141,218.984403
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,478.0,3.016736,158.449376
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,170.0,2.729412,62.284483
4,5,Father of the Bride Part II (1995),Comedy,1995,296.0,3.006757,98.444944


**Now we can decide which film is the higest rated film based on it's rank**

In [54]:
#movies_agg.iloc[movies_agg['rank'].idxmax()]
movies_agg.sort_values('rank',ascending=False).iloc[0] # grab the higest ranked film with a sort.

movie_id                         2858
title          American Beauty (1999)
genres                   Comedy|Drama
year                             1999
num_ratings                      3428
mean_rating                   4.31739
rank                          793.999
Name: 2788, dtype: object

### Now we can start working out what the best date night film is

*Of course the model for determining such a film will be heavily biased towards the creators own notions.*

---

Now that we have ranked films according to their ratings and number of ratings, we have to state some assumptions;

**The reccommendation model assumes:**
1. a date night is between two people.
2. Those two people have records in our users dataset, that is they have rated film

#### So how do we decide what film to show a couple?

Given two people on a date the minimal amount of information we need from them is their age and sex and user_id.
From this we can check our model (it's going to be a simple one) to see which films would appeal the best, and then
select the one with the highest rank. Additionally we can utilize information from ratings that members of the couple may
have given to bias the film selection towards generes they have rated highly.

***We need to develop a system*** which we can treat as a function that takes a tuple of user info and spits out a single
film.

$$f(X,Y) \rightarrow Z$$
Where $X$ and $Y$ are sets.

**EX:** 
$$f(\{Films, Ratings\},\{Genre, Userid, Age, Sex, [Ratings]\}, \{Genre, Userid, Age, Sex, [Ratings]\}) \rightarrow Film$$

We keep it really simple. Given a dataframe of movies left joined with their ratings, an aggregation of the films, which 
has the rank and meam_rating of each film, and a couple, we compute each memeber of the couple's most rated genre(s) and 
their mean rating across all ratings they have made. Then we simply use this information to produce a temproary dataframe
which has the films that have at least the same genre as each member's most rated genre. Finally we select from the temp
dataframe movies which have mean rating that is at least 90% numerically similar to the mean rating of each member of the couple, and then we choose the film from this that has the highest rank.

In [51]:
def numeric_similarity(x, y):
    return min(x,y)/max(x,y)

def is_similar(x, y, thresh=0.5):
    s = []
    for i in x:
        s.append((numeric_similarity(i,y) > thresh))
    return s

def recommend_film(films, couple):
    '''
    @param films - Dataframe of movies left merged with their ratings.
    @param couple - Dataframe of two users of the "site/system".
    
    @returns Pandas.Series film - A series object containg info on the recommended film.
        Unless a film could not be determined, in which case, None is returned.
    '''
    # get the most rated genre of both 
    c1 = {
        'most_rated_genre':'',
        'mean_rating':0
    }

    c2 = {
        'most_rated_genre':'',
        'mean_rating':0
    }

    c1['most_rated_genre'] = \
        movies_ratings[movies_ratings['user_id'] == couple.iloc[0]['user_id']]['genres'].value_counts().index.tolist()[0]
    c1['mean_rating'] = \
        movies_ratings[movies_ratings['user_id'] == couple.iloc[0]['user_id']]['rating'].mean()
    c2['most_rated_genre'] = \
        movies_ratings[movies_ratings['user_id'] == couple.iloc[1]['user_id']]['genres'].value_counts().index.tolist()[0]
    c2['mean_rating'] = \
        movies_ratings[movies_ratings['user_id'] == couple.iloc[1]['user_id']]['rating'].mean()
    
    mask = movies_agg['genres'].str.contains(c1['most_rated_genre']) & \
        movies_agg['genres'].str.contains(c2['most_rated_genre'])
    
    print(c1,c2)
    tmp = movies_agg[mask]
    tmp = tmp[is_similar(tmp['mean_rating'],c1['mean_rating'],0.9)]
    tmp = tmp[is_similar(tmp['mean_rating'],c2['mean_rating'],0.9)]
    tmp.reset_index(inplace=True)
    # now select the movie from this which has the highest rank
    if tmp['rank'].size == 0:
        #print('Sorry, were having trouble finding a good movie for you.')
        return None
    else:
        #print(tmp.iloc[tmp['rank'].idxmax()])
        return tmp.sort_values('rank',ascending=False).iloc[0]
    
couple = users.sample(n=2)
recommend_film(movies_ratings, couple)

{'most_rated_genre': 'Action|Adventure', 'mean_rating': 3.7115384615384617} {'most_rated_genre': 'Horror', 'mean_rating': 3.9646464646464645}


index                   1365
movie_id                1387
title            Jaws (1975)
genres         Action|Horror
year                    1975
num_ratings             1697
mean_rating          4.08957
rank                 414.958
Name: 2, dtype: object

### This approach has some flaws:
---
1. There are cases in which a film that suits the couple cannot be found.
2. It assumes that each member of the couple has made ratings on films in the system.
    * A possible solution to this would be to prompt the couple to choose their favorite genres, respectively
3. It does not consider things such as gender, highest rated year of each member, or timestamp.
    * These are all things that could produce better recommendations.
4. It's not an established method of recommending things.
    * There are plenty of recommendation algorithms out there, that utilize better measures of similarity, etc. 
    * For the sake of time, I made my own methedology up.

## Part 2

Load the dataset in `titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [2]:
from IPython.core.display import HTML
HTML(filename='../data/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


In [138]:
# you would need xlrd - pip install xlrd
t_file = pd.ExcelFile('../data/titanic.xls')
t_df = t_file.parse("titanic", header=0)
t_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### Women and children first?

*** 1. Use the `groupby` method to calculate the proportion of passengers that survived by sex. *** 

*** 2. Calculate the same proportion, but by class and sex. *** 

*** 3. Create age categories: children (under 14 years), adolescents (14-20), adult (21-64), and senior(65+), and calculate survival proportions by age category, class and sex. ***

In [134]:
# note to self; use better variable names...
g = t_df.groupby(['survived','sex'])
s = g['survived'].sum()

s_f, s_m = s[[2,3]]
n_survived = s_f + s_m

print(n_survived)

# The total number of male passengers
n_male = t_df[t_df['sex'] == 'male']['sex'].value_counts()[0]
n_female = t_df[t_df['sex'] == 'female']['sex'].value_counts()[0]
print(n_male)
print(n_female)
stats_sex = {
    'proportion_male_survied':       s_m / n_male,
    'proportion_fmale_survived':     s_f / n_female,
    'proportion_male_survived_tot':  s_f / n_survived,
    'proportion_fmale_survived_tot': s_m / n_survived
}
stats_sex

500
843
466


{'proportion_fmale_survived': 0.72746781115879833,
 'proportion_fmale_survived_tot': 0.32200000000000001,
 'proportion_male_survied': 0.19098457888493475,
 'proportion_male_survived_tot': 0.67800000000000005}

***339 women survived*** and ***161 men survived***. That means that ***500*** out of ***1309*** passengers *survived*.
This is of course not counting the crew members. A quick google search shows us some other numbers to compare against.

According to the search, there were ***1316 passengers*** and ***498*** of them survived. This isn't too far off from what our dataset tells us.

The proportion of the survivors that were men compared to all survivors is

$$\frac{161_{men}}{500_{survivors}} = 0.32$$

The proportion of the survivors that were women compared to all survivors is

$$\frac{339_{women}}{500_{survivors}} = 0.68$$

The proportions of men that survived out of all the male passengers is

$$\frac{161_{men}}{843_{men}} = 0.19$$ 

The proportions of women that survived out of all the female passengers is

$$\frac{339_{women}}{466_{women}} = 0.73$$

So, of the $500$ survivors, $32\%$ were **male** and $68\%$ were **female**.

In [132]:
# note to self; use better variable names...
g = t_df.groupby(['survived','pclass'])
s = g['survived'].sum()

n_s_first, n_s_second, n_s_third = s.values[[3,4,5]]
print(n_s_first)
print(n_s_second)
print(n_s_third)
tot_s = n_s_first + n_s_second + n_s_third

# should be 500 in total
print(tot_s == 500)
print(s)

stats_class = {
    'proportion_first_survied':   n_s_first  / tot_s,
    'proportion_second_survived': n_s_second / tot_s,
    'proportion_third_survived':  n_s_third  / tot_s,
}

stats_class

200
119
181
True
survived  pclass
0         1           0
          2           0
          3           0
1         1         200
          2         119
          3         181
Name: survived, dtype: int64


{'proportion_first_survied': 0.40000000000000002,
 'proportion_second_survived': 0.23799999999999999,
 'proportion_third_survived': 0.36199999999999999}

### Proportions of survivors by class 

$$first\_class = \frac{200_{first}}{500_{survivors}} = 0.40 = 40\%$$

$$second\_class = \frac{119_{second}}{500_{survivors}} = 0.24 = 24\%$$

$$third\_class = \frac{181_{first}}{500_{survivors}} = 0.36 = 36\%$$


In [139]:
# Create age categories: children (under 14 years), adolescents (14-20), adult (21-64), and senior(65+), 
# and calculate survival proportions by age category, class and sex.

def categorize_age(age):
    if age < 14: return 'child'
    if 14 <= age <= 20: return 'adolescent'
    if 21 <= age <= 64: return 'adult'
    if age >= 65: return 'senior'
    
t_df['age_category'] = t_df['age'].apply(categorize_age)

In [140]:
t_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,age_category
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",adult
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",child
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",child
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",adult
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",adult


In [332]:
g = t_df.groupby(['age_category','pclass','sex'])

def proportion_survived(x):
    return x.count() / tot_s

n_df = g['survived'].agg([proportion_survived]).reset_index()
n_df

,age_category,pclass,sex,proportion_survived
0,adolescent,1,female,0.030
1,adolescent,1,male,0.010
2,adolescent,2,female,0.026
3,adolescent,2,male,0.034
4,adolescent,3,female,0.070
5,adolescent,3,male,0.128
6,adult,1,female,0.232
7,adult,1,male,0.268
8,adult,2,female,0.152
9,adult,2,male,0.256


### That's a lot of proportions...
---
As can be seen above there are 22 different proportions that we calculated. Each one is a function of $age\_category, pclass, sex, survived$. For example the first entry in the above dataframe would be read as 'The proportion of survivors that were adolescent first class females is $0.030$.

Each proportion is simply 

$$\frac{count\_of\_survivor\_type}{total\_num\_survivors}$$